# Unique species
Visualize the numbers of unique species within and across kingdoms.

In [12]:
import json
from collections import defaultdict

## Read in & preprocess data

In [15]:
with open('../data/citation_network/intermediate_saves/classmains_07Jan2024/species_dict.json') as myf:
    species_dict = json.load(myf)

In [4]:
with open('../data/citation_network/intermediate_saves/classmains_07Jan2024/species_ids.json') as myf:
    species_ids = json.load(myf)

In [9]:
# Cast all NCBI Taxonomy IDs to integers
species_ids = {k: int(v) for k, v in species_ids.items()}

## Numbers of unique species
### Entire dataset

In [10]:
total_uniq = set(species_ids.values())
print(f'There are {len(total_uniq)} unique species mentions in the overall dataset.')

There are 4809 unique species mentions in the overall dataset.


### Per class

In [14]:
specs_per_cls = defaultdict(list)
for spec, cls in species_dict.items():
    print(cls)
    specs_per_cls[cls].append(species_ids[spec])
specs_per_cls = {k: list(set(v)) for k, v in specs_per_cls.items()}
for cls, ids in specs_per_cls.items():
    print(f'There are {len(ids)} unique species in class {cls}')

Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
Microbe
